<div style="background-color: #e8f5e9;
            border-left: 4px solid #81c784; /* Lighter green, thinner border */
            padding: 15px;
            border-radius: 8px; /* Slightly more rounded */
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1); /* Subtle shadow */
            animation: fadeIn 0.8s ease-out;">
<style>
  @keyframes fadeIn {
    0% { opacity: 0; transform: translateY(-5px); }
    100% { opacity: 1; transform: translateY(0); }
  }
</style>

# 🔬 Analyse Micro du Marché Immobilier (Suite DVF 2021-2025)

**Objectif :** Ce notebook est la **troisième étape** de notre analyse, faisant suite à `visualisation.ipynb` et `analyse_strategique.ipynb`. Il se concentre sur le niveau "Micro" : l'analyse détaillée au niveau des **communes**.

**Méthodologie (Micro) :**
* **1. Configuration & Nettoyage :** Import des librairies, redéfinition de la fonction `clean_dvf_data` et de `load_specific_cities_data`.
* **2. Analyse Comparative (Besoin I.2) :** Comparaison des dynamiques de prix entre les grandes métropoles chargées.
* **3. Fiche d'Identité Commune (Besoin III.5 / II.1) :** Fonction pour générer un résumé statistique et graphique pour une commune spécifique (via son code INSEE).
* **4. Cartographie Locale (Besoin IV.2 - Heatmap) :** Visualisation des "points chauds" de prix au sein d'une ville via une carte thermique.
* **5. Rapport PDF (Besoin IV.1) :** Fonction simple pour générer un PDF synthétique pour une commune.
* **6. Conclusion & Perspectives :** Synthèse et pistes pour intégrer des données externes (loyers, démographie - Besoins I.5, III.2).

</div>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff # Pour les heatmaps
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, FileLink
import warnings
import os
from tqdm.auto import tqdm
import gc
from datetime import datetime
from fpdf import FPDF # Pour la génération PDF

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
px.defaults.template = "plotly_white"

print("✅ Bibliothèques importées.")

<div style="background-color: #e8f5e9;
            border-left: 4px solid #81c784; /* Lighter green, thinner border */
            padding: 15px;
            border-radius: 8px; /* Slightly more rounded */
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1); /* Subtle shadow */
            animation: fadeIn 0.8s ease-out;">
<style>
  @keyframes fadeIn {
    0% { opacity: 0; transform: translateY(-5px); }
    100% { opacity: 1; transform: translateY(0); }
  }
</style>

### 🧹 1.1 Redéfinition des Fonctions de Chargement et Nettoyage

**Rappel :** Ces fonctions sont identiques à celles des notebooks précédents pour garantir la cohérence des données. `load_specific_cities_data` est essentielle ici car nous ne travaillons que sur des sous-ensembles

In [ ]:
# --- Copie de la fonction clean_dvf_data ---
def clean_dvf_data(df):
    """
    Fonction de nettoyage optimisée pour les données DVF (identique aux notebooks précédents).
    """
    if df.empty: return pd.DataFrame()
    print("   -> Application du nettoyage standard...")
    initial_rows = len(df)
    # 1. Conversion de date
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
    # 2. Nettoyage numériques et suppression lignes critiques manquantes
    cols_to_numeric = ['valeur_fonciere', 'surface_reelle_bati', 'nombre_pieces_principales']
    for col in cols_to_numeric:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
    cols_critiques = ['date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 'latitude', 'longitude', 'type_local', 'code_departement', 'code_commune', 'nom_commune']
    cols_a_verifier = [col for col in cols_critiques if col in df.columns]
    df = df.dropna(subset=cols_a_verifier)
    # 3. Filtrage aberrations valeur/surface/pièces
    df = df[df['valeur_fonciere'].between(1000, 20_000_000)]
    df = df[df['surface_reelle_bati'].between(10, 1000)]
    df = df[df['nombre_pieces_principales'].between(1, 20)]
    # 4. Filtre type de bien
    df = df[df['type_local'].isin(['Appartement', 'Maison'])] # Focus micro
    # 5. Calcul Prix m² et filtrage outliers
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    p_01 = df['prix_m2'].quantile(0.01)
    p_99 = df['prix_m2'].quantile(0.99)
    df = df[df['prix_m2'].between(max(100, p_01), min(30000, p_99))]
    # 6. Features temporelles
    df['annee'] = df['date_mutation'].dt.year
    df['trimestre'] = df['date_mutation'].dt.to_period('Q').astype(str) # Pour Plotly
    df['mois'] = df['date_mutation'].dt.month

    final_rows = len(df)
    print(f"   -> Nettoyage terminé. {initial_rows - final_rows:,} lignes écartées. {final_rows:,} transactions valides.")
    return df

# --- Copie de la fonction load_specific_cities_data ---
BASE_CSV_PATH = './' # Ajuster si nécessaire
FILE_NAMES_CSV = [f"dvf_geolocalisees_{year}.csv" for year in range(2021, 2026)]
FILE_PATHS_CSV = [os.path.join(BASE_CSV_PATH, f) for f in FILE_NAMES_CSV if os.path.exists(os.path.join(BASE_CSV_PATH, f))]

# Colonnes minimales pour l'analyse micro + nettoyage
COLS_FOR_CITY = ['date_mutation', 'valeur_fonciere', 'code_commune', 'nom_commune', 'code_departement',
                 'surface_reelle_bati', 'nombre_pieces_principales', 'type_local', 'latitude', 'longitude']
DTYPES_FOR_CITY = {'valeur_fonciere': 'float32', 'code_commune': 'str', 'nom_commune': 'str', 'code_departement': 'str',
                   'surface_reelle_bati': 'float32', 'nombre_pieces_principales': 'float32',
                   'type_local': 'category', 'latitude': 'float32', 'longitude': 'float32'}

def load_specific_cities_data(city_codes, file_paths=FILE_PATHS_CSV, use_cols=COLS_FOR_CITY, dtypes=DTYPES_FOR_CITY):
    """ Charge et nettoie les données pour une liste de codes communes spécifiques. """
    print(f"🔬 Chargement chirurgical des données pour les communes : {city_codes}...")
    all_city_chunks = []
    if not file_paths: print("❌ Aucun fichier CSV source trouvé."); return pd.DataFrame()

    city_codes_str = [str(c) for c in city_codes] # Assurer la comparaison str vs str

    for file_path in tqdm(file_paths, desc="Lecture CSV"):
        try:
            chunk_iter = pd.read_csv(file_path, usecols=lambda col: col in use_cols, dtype=dtypes,
                                     chunksize=100_000, low_memory=False, on_bad_lines='skip')
            for chunk in chunk_iter:
                # Filtrer directement sur le code commune (converti en str pour le chunk aussi)
                chunk_filtered = chunk[chunk['code_commune'].astype(str).isin(city_codes_str)]
                if not chunk_filtered.empty: all_city_chunks.append(chunk_filtered)
        except FileNotFoundError: print(f"\n⚠️ Fichier ignoré: {file_path}")
        except Exception as e: print(f"\n⚠️ Erreur lecture {file_path}: {e}")

    if not all_city_chunks: print("❌ Aucune donnée trouvée."); return pd.DataFrame()

    print("   -> Concaténation...")
    df_cities_raw = pd.concat(all_city_chunks, ignore_index=True)
    print(f"   -> {len(df_cities_raw):,} transactions brutes chargées.")

    df_cities_clean = clean_dvf_data(df_cities_raw)

    # Optimisation mémoire finale
    print("   -> Optimisation mémoire finale...")
    for col in df_cities_clean.select_dtypes(include=['object', 'category']).columns:
         if col not in ['trimestre']: # Trimestre reste object pour Plotly animation
            df_cities_clean[col] = df_cities_clean[col].astype('category')

    gc.collect()
    print(f"✅ Chargement et nettoyage terminés.")
    df_cities_clean.info(memory_usage='deep')
    return df_cities_clean

print("✅ Fonctions utilitaires définies.")

<div style="background: linear-gradient(to right, #e8f5e9, #f1f8e9); /* Soft green gradient */
            border-left: 4px solid #a5d6a7; /* Lighter, thinner green border */
            padding: 18px 20px; /* Slightly adjusted padding */
            border-radius: 8px; /* Softer corners */
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); /* Softer shadow */
            margin-bottom: 20px; /* Add some space below */
            animation: slideIn 0.7s ease-out;">
<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  /* Optional: Adjust heading style slightly if needed */
  .custom-div h2 {
      color: #388e3c; /* Darker green for heading contrast */
      margin-bottom: 12px;
  }
  .custom-div p {
      line-height: 1.6;
  }
</style>

<div class="custom-div"> <h2>🏙️ 2. Analyse Comparative : Métropoles vs. Villes Moyennes (Besoin I.2)</h2>
    <p><strong>Objectif :</strong> Comparer la dynamique des prix (€/m²) entre les grandes métropoles (Lyon, Marseille, Bordeaux, Lille, Nantes) déjà chargées dans <code>df_micro_example</code> (si le notebook précédent a été exécuté) ou rechargées ici.</p>
    <p><strong>Méthode :</strong> Nous allons tracer l'évolution du prix médian/m² par trimestre pour ces 5 villes sur un même graphique.</p>
</div>

</div>

In [ ]:
# --- Recharger ou réutiliser les données des métropoles ---
METROPOLE_CODES = [str(c) for c in range(69381, 69390)] + \
                  [str(c) for c in range(13201, 13217)] + \
                  ['33063', '59350', '44109'] # Lyon, Marseille, Bordeaux, Lille, Nantes

# Essayer de récupérer depuis le notebook précédent si exécuté dans le même environnement
if 'df_micro_example' in locals() and not df_micro_example.empty and \
   set(df_micro_example['code_commune'].unique()) == set(METROPOLE_CODES):
    print("Réutilisation des données 'df_micro_example' du notebook précédent.")
    df_metropoles = df_micro_example.copy()
else:
    print("Chargement des données pour les métropoles...")
    df_metropoles = load_specific_cities_data(METROPOLE_CODES)

# Fonction pour l'analyse comparative
def plot_metropole_comparison(df):
    if df.empty:
        print("❌ Aucune donnée de métropole à comparer.")
        return

    print("📊 Comparaison de l'évolution des prix (€/m²) par métropole...")

    # Simplifier le nom de la ville (pour Marseille et Lyon)
    def simplify_city_name(row):
        if row['code_commune'].startswith('6938'): return 'Lyon'
        if row['code_commune'].startswith('132'): return 'Marseille'
        return row['nom_commune']
    df['ville_simple'] = df.apply(simplify_city_name, axis=1).astype('category')

    # Agréger par ville et trimestre
    df_agg = df.groupby(['ville_simple', 'trimestre'])['prix_m2'].median().reset_index()

    # Trier les trimestres correctement pour le graphique
    df_agg['trimestre_dt'] = pd.to_datetime(df_agg['trimestre'].str.replace('Q', '-Q'))
    df_agg = df_agg.sort_values('trimestre_dt')

    # --- Visualisation ---
    fig = px.line(
        df_agg,
        x='trimestre',
        y='prix_m2',
        color='ville_simple',
        markers=True,
        title="<b>🏙️ Évolution Comparée du Prix Médian (€/m²) - Métropoles (Trimestriel)</b>",
        labels={'trimestre': 'Trimestre', 'prix_m2': 'Prix Médian (€/m²)', 'ville_simple': 'Ville'},
        height=600
    )
    fig.update_layout(title_x=0.5, legend_title_text='Métropole')
    fig.update_traces(marker=dict(size=8))
    fig.show()

# --- Exécution ---
if not df_metropoles.empty:
    plot_metropole_comparison(df_metropoles)
else:
    print("❌ Impossible de comparer les métropoles (données non chargées).")

<div style="background: linear-gradient(to right, #f3e5f5, #fce4ec); /* Soft purple/pink gradient */
            border-left: 4px solid #ce93d8; /* Lighter, thinner purple border */
            padding: 18px 20px;
            border-radius: 8px;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); /* Softer shadow */
            margin-bottom: 20px;
            animation: slideIn 0.7s ease-out;">
<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  .custom-div-purple h2 { /* Class for this specific color scheme */
      color: #7b1fa2; /* Darker purple for heading contrast */
      margin-bottom: 12px;
  }
  .custom-div-purple p, .custom-div-purple ul { /* Target paragraphs and lists */
      line-height: 1.6;
  }
  .custom-div-purple ul {
      padding-left: 20px; /* Indent list */
      margin-top: 10px;
  }
  .custom-div-purple li {
      margin-bottom: 5px; /* Spacing between list items */
  }
</style>

<div class="custom-div-purple"> <h2>📄 3. Fiche d'Identité par Commune (Besoin III.5 / II.1)</h2>
    <p><strong>Objectif :</strong> Créer une fonction <code>generate_city_factsheet</code> qui prend un code commune en entrée et affiche une fiche synthétique :</p>
    <ul>
        <li>KPIs clés (Prix médian actuel, Volume 5 ans, Évolution 5 ans).</li>
        <li>Graphique d'évolution trimestrielle du prix médian/m².</li>
        <li>Graphique de répartition des ventes (Maisons vs Appartements).</li>
        <li><em>Bonus (Simulation Besoin II.1)</em>: Potentiellement montrer la distribution des prix pour des biens similaires (ex: appartements 3 pièces).</li>
    </ul>
</div>

</div>

In [ ]:
def generate_city_factsheet(code_commune, df_source=None):
    """
    Génère et affiche une fiche synthétique pour une commune donnée.
    Charge les données si elles ne sont pas fournies via df_source.
    """
    print(f"--- Génération de la Fiche pour la Commune {code_commune} ---")

    # Charger les données si nécessaire
    if df_source is None or df_source.empty or code_commune not in df_source['code_commune'].unique():
        print(f"Chargement des données pour {code_commune}...")
        df_city = load_specific_cities_data([code_commune])
        if df_city.empty:
            print(f"❌ Aucune donnée trouvée pour la commune {code_commune}.")
            return
    else:
        # Filtrer le DataFrame source
        df_city = df_source[df_source['code_commune'] == code_commune].copy()
        if df_city.empty:
            print(f"❌ Aucune donnée trouvée pour {code_commune} dans le DataFrame fourni.")
            return
        print(f"Utilisation des données pré-chargées pour {code_commune}.")

    nom_commune = df_city['nom_commune'].iloc[0]
    start_year = df_city['annee'].min()
    end_year = df_city['annee'].max()

    # --- 1. KPIs Clés ---
    kpi_output = widgets.Output()
    with kpi_output:
        print(f"\n**📊 Indicateurs Clés pour {nom_commune} ({code_commune}) [{start_year}-{end_year}]**")
        total_ventes = len(df_city)
        print(f"- Volume total de ventes (5 ans) : {total_ventes:,}")

        if total_ventes > 10: # Seuil minimum pour stats fiables
            prix_median_actuel = df_city[df_city['annee'] == end_year]['prix_m2'].median()
            print(f"- Prix médian/m² ({end_year}) : {prix_median_actuel:,.0f} €")

            if start_year != end_year:
                prix_median_debut = df_city[df_city['annee'] == start_year]['prix_m2'].median()
                if prix_median_debut > 0:
                    evolution_5ans = ((prix_median_actuel - prix_median_debut) / prix_median_debut) * 100
                    print(f"- Évolution prix/m² ({start_year}-{end_year}) : {evolution_5ans:+.1f}%")
                else: print("- Évolution prix/m² : N/A (prix début nul)")
            else: print("- Évolution prix/m² : N/A (une seule année)")
        else:
            print("- KPIs détaillés : N/A (volume de ventes trop faible)")

    # --- 2. Graphique Évolution Prix Trimestriel ---
    evo_output = widgets.Output()
    with evo_output:
        if total_ventes > 10:
            df_agg_trim = df_city.groupby('trimestre')['prix_m2'].median().reset_index()
            df_agg_trim['trimestre_dt'] = pd.to_datetime(df_agg_trim['trimestre'].str.replace('Q', '-Q'))
            df_agg_trim = df_agg_trim.sort_values('trimestre_dt')

            fig_evo = px.line(df_agg_trim, x='trimestre', y='prix_m2', markers=True,
                              title=f"Évolution Prix Médian (€/m²) à {nom_commune}",
                              labels={'trimestre': 'Trimestre', 'prix_m2': 'Prix Médian (€/m²)'})
            fig_evo.update_layout(title_x=0.5)
            fig_evo.show()
        else: print("\nGraphique Évolution : N/A (volume trop faible)")

    # --- 3. Graphique Répartition Type ---
    type_output = widgets.Output()
    with type_output:
        if total_ventes > 0:
            type_counts = df_city['type_local'].value_counts()
            fig_type = go.Figure(data=[go.Pie(labels=type_counts.index, values=type_counts.values, hole=.3,
                                             title=f"Répartition Ventes (Maison/Appart.) à {nom_commune}",
                                             marker=dict(colors=['#1f77b4', '#ff7f0e']))])
            fig_type.update_layout(title_x=0.5)
            fig_type.show()
        else: print("\nGraphique Répartition : N/A")

    # --- 4. Bonus: Simulation Estimation (II.1) ---
    # Pour un type de bien courant (ex: Appartement 3 pièces)
    sim_output = widgets.Output()
    with sim_output:
        df_sim = df_city[(df_city['type_local'] == 'Appartement') & (df_city['nombre_pieces_principales'] == 3)]
        if len(df_sim) > 5: # Seuil pour distribution
            fig_sim = px.histogram(df_sim, x='prix_m2', nbins=20, marginal="box",
                                   title=f"Distribution Prix/m² - Appartements 3 pièces ({nom_commune}, {end_year})",
                                   labels={'prix_m2': 'Prix au m² (€)'})
            median_sim = df_sim['prix_m2'].median()
            fig_sim.add_vline(x=median_sim, line_dash="dash", line_color="red", annotation_text=f"Médiane: {median_sim:,.0f}€")
            fig_sim.update_layout(title_x=0.5)
            fig_sim.show()
        else: print("\nSimulation Estimation (Appt 3p) : N/A (pas assez de transactions similaires)")


    # --- Affichage Final avec Widgets ---
    tab = widgets.Tab()
    tab.children = [kpi_output, evo_output, type_output, sim_output]
    tab.titles = ['KPIs Clés', 'Évolution Prix', 'Répartition Type', 'Estimation (Exemple)']
    display(tab)


# <div style="background-color: #f0fff4; border-left: 6px solid #4CAF50; padding: 15px; border-radius: 5px;">
#
# ### ▶️ Exécution : Test de la Fiche Commune
#
# Testons la fonction sur une commune, par exemple **Villeurbanne (69100)** ou **Bordeaux (33063)**.
#
# </div>

# %%
# --- Test sur Villeurbanne ---
CODE_COMMUNE_TEST = '69100' # Ou '33063' pour Bordeaux, etc.

# Option 1: Utiliser les données des métropoles si déjà chargées et contenant le code
if not df_metropoles.empty and CODE_COMMUNE_TEST in df_metropoles['code_commune'].unique():
    generate_city_factsheet(CODE_COMMUNE_TEST, df_source=df_metropoles)
# Option 2: Charger spécifiquement les données
else:
    generate_city_factsheet(CODE_COMMUNE_TEST)

<div style="background: linear-gradient(to right, #e0f7fa, #e0f2f7); /* Soft cyan gradient */
            border-left: 4px solid #80deea; /* Lighter, thinner cyan border */
            padding: 18px 20px;
            border-radius: 8px;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); /* Softer shadow */
            margin-bottom: 20px;
            animation: slideIn 0.7s ease-out;">
<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  .custom-div-cyan h2 { /* Class for this specific color scheme */
      color: #0097a7; /* Darker cyan for heading contrast */
      margin-bottom: 12px;
  }
  .custom-div-cyan p {
      line-height: 1.6;
  }
</style>

<div class="custom-div-cyan"> <h2>🔥 4. Cartographie Locale : Heatmap des Prix (Besoin IV.2)</h2>
    <p><strong>Objectif :</strong> Visualiser les zones les plus chères ("points chauds") au sein d'une commune via une carte thermique (heatmap).</p>
    <p><strong>Méthode :</strong> Nous utilisons <code>plotly.figure_factory.create_densitymapbox</code>. Cette fonction nécessite les latitudes et longitudes des transactions et une valeur (ici, <code>prix_m2</code>) pour déterminer l'intensité de la chaleur.</p>
</div>

</div>

In [ ]:
import plotly.graph_objects as go # Make sure go is imported
import plotly.figure_factory as ff # Keep ff if used elsewhere, otherwise remove

def plot_city_heatmap(code_commune, df_source=None):
    """
    Génère une heatmap des prix/m² pour une commune donnée.
    CORRIGÉ pour utiliser go.Densitymapbox.
    """
    print(f"--- Génération de la Heatmap pour la Commune {code_commune} ---")

    # Charger/Filtrer les données
    if df_source is None or df_source.empty or code_commune not in df_source['code_commune'].unique():
        print(f"Chargement des données pour {code_commune}...")
        df_city = load_specific_cities_data([code_commune])
        if df_city.empty: print(f"❌ Aucune donnée pour {code_commune}."); return
    else:
        df_city = df_source[df_source['code_commune'] == code_commune].copy()
        if df_city.empty: print(f"❌ Aucune donnée pour {code_commune} dans le DataFrame fourni."); return
        print(f"Utilisation des données pré-chargées pour {code_commune}.")

    nom_commune = df_city['nom_commune'].iloc[0]

    if len(df_city) < 10:
        print("❌ Pas assez de transactions pour générer une heatmap pertinente.")
        return

    center_lat = df_city['latitude'].mean()
    center_lon = df_city['longitude'].mean()

    # --- Création de la Heatmap (CORRIGÉ) ---
    print("🔥 Génération de la carte thermique...")
    fig = go.Figure(go.Densitymapbox(
        lat=df_city['latitude'],
        lon=df_city['longitude'],
        z=df_city['prix_m2'],
        radius=10,
        coloraxis='coloraxis' # Important: Links trace to coloraxis in layout
    ))

    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_center={"lat": center_lat, "lon": center_lon},
        mapbox_zoom=11,
        coloraxis_showscale=True, # Display color bar
        coloraxis_colorbar_title='Prix/m² (€)', # Color bar title
        title=f'<b>🔥 Heatmap des Prix Immobiliers (€/m²) - {nom_commune} ({code_commune})</b>',
        title_x=0.5,
        height=700
    )
    # ------------------------------------

    fig.show()

# %% [markdown]
# <div style="background-color: #f0fff4; border-left: 6px solid #4CAF50; padding: 15px; border-radius: 5px;">
#
# ### ▶️ Exécution : Test de la Heatmap
#
# Testons sur la même commune que précédemment.
#
# </div>

# %%
# --- Test Heatmap ---
if not df_metropoles.empty and CODE_COMMUNE_TEST in df_metropoles['code_commune'].unique():
    plot_city_heatmap(CODE_COMMUNE_TEST, df_source=df_metropoles)
else:
    plot_city_heatmap(CODE_COMMUNE_TEST) # Charge les données si besoin

<div style="background: linear-gradient(to right, #f5f5f5, #fafafa); /* Light grey/beige gradient */
            border-left: 4px solid #bcaaa4; /* Lighter brown/beige border */
            padding: 18px 20px;
            border-radius: 8px;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); /* Softer shadow */
            margin-bottom: 20px;
            animation: slideIn 0.7s ease-out;">
<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  .custom-div-brown h2 { /* Class for this specific color scheme */
      color: #5d4037; /* Darker brown for heading contrast */
      margin-bottom: 12px;
  }
  .custom-div-brown p {
      line-height: 1.6;
  }
  .custom-div-brown .note { /* Style for the note */
      font-style: italic;
      font-size: 0.9em;
      color: #616161; /* Slightly muted color for the note */
      margin-top: 10px;
  }
</style>

<div class="custom-div-brown"> <h2>📑 5. Génération de Rapport PDF (Besoin IV.1)</h2>
    <p><strong>Objectif :</strong> Créer une fonction <code>generate_pdf_report</code> qui génère un PDF simple contenant les KPIs et éventuellement

In [ ]:
# --- Corrected PDF Generation Function ---

class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        # Use UTF-8 encoding for strings passed to cell
        self.cell(0, 10, 'Rapport d\'Analyse Immobilière - Commune', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        # Use UTF-8 encoding
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(4)

    def chapter_body(self, body):
        self.set_font('Arial', '', 10)
        # Use UTF-8 encoding
        self.multi_cell(0, 5, body)
        self.ln()

    def add_kpi(self, key, value):
        self.set_font('Arial', '', 10)
        # Use UTF-8 encoding
        self.cell(60, 7, f"- {key} :", 0, 0, 'L')
        self.set_font('Arial', 'B', 10)
        # Use UTF-8 encoding and ensure value is a string
        self.cell(0, 7, str(value), 0, 1, 'L')

def generate_pdf_report(code_commune, df_source=None):
    """ Génère un rapport PDF simple pour une commune (corrigé pour encodage). """
    print(f"--- Génération du Rapport PDF pour la Commune {code_commune} ---")

    # Charger/Filtrer les données (Same as before)
    if df_source is None or df_source.empty or code_commune not in df_source['code_commune'].unique():
        print(f"Chargement des données pour {code_commune}...")
        df_city = load_specific_cities_data([code_commune])
        if df_city.empty: print(f"❌ Aucune donnée pour {code_commune}."); return None
    else:
        df_city = df_source[df_source['code_commune'] == code_commune].copy()
        if df_city.empty: print(f"❌ Aucune donnée pour {code_commune} dans le DataFrame fourni."); return None
        print(f"Utilisation des données pré-chargées pour {code_commune}.")

    nom_commune = df_city['nom_commune'].iloc[0]
    start_year = df_city['annee'].min()
    end_year = df_city['annee'].max()
    total_ventes = len(df_city)

    # Création du PDF
    pdf = PDFReport()
    pdf.add_page()

    # Section Titre (Ensure UTF-8 compatibility for commune names if needed)
    pdf.set_font('Arial', 'B', 14)
    # FPDF handles UTF-8 automatically if fonts support it, ensure strings are standard Python strings
    pdf.cell(0, 10, f"Synthèse Immobilière - {nom_commune} ({code_commune})", 0, 1, 'C')
    pdf.set_font('Arial', '', 10)
    pdf.cell(0, 7, f"Période d'analyse : {start_year} - {end_year}", 0, 1, 'C')
    pdf.ln(10)

    # Section KPIs
    # CORRECTION: Removed emoji 📊
    pdf.chapter_title('Indicateurs Clés')
    pdf.add_kpi('Volume total de ventes', f"{total_ventes:,}")
    if total_ventes > 10:
        prix_median_actuel = df_city[df_city['annee'] == end_year]['prix_m2'].median()
        # CORRECTION: Replaced € with EUR
        pdf.add_kpi(f'Prix médian/m² ({end_year})', f"{prix_median_actuel:,.0f} EUR")
        if start_year != end_year:
            prix_median_debut = df_city[df_city['annee'] == start_year]['prix_m2'].median()
            if prix_median_debut > 0:
                evolution_5ans = ((prix_median_actuel - prix_median_debut) / prix_median_debut) * 100
                pdf.add_kpi(f'Évolution prix/m² ({start_year}-{end_year})', f"{evolution_5ans:+.1f}%")
            else: pdf.add_kpi('Évolution prix/m²', "N/A (prix début nul)")
        else: pdf.add_kpi('Évolution prix/m²', "N/A (une seule année)")

        type_counts = df_city['type_local'].value_counts(normalize=True) * 100
        pdf.add_kpi('Part Appartements', f"{type_counts.get('Appartement', 0):.1f}%")
        pdf.add_kpi('Part Maisons', f"{type_counts.get('Maison', 0):.1f}%")
    else:
        pdf.chapter_body("KPIs détaillés non disponibles (volume de ventes trop faible).")

    # Section Graphiques Placeholder (Same as before)
    pdf.ln(10)
    # CORRECTION: Removed emoji 📈
    pdf.chapter_title('Visualisations (Placeholder)')
    pdf.chapter_body("Les graphiques d'évolution des prix et de répartition des types de biens peuvent être ajoutés ici (nécessite sauvegarde préalable en image).")

    # Sauvegarde du PDF
    pdf_filename = f"Rapport_Immobilier_{code_commune}_{datetime.now().strftime('%Y%m%d')}.pdf"
    try:
        # FPDF's output method handles encoding correctly for standard fonts
        pdf.output(pdf_filename)
        print(f"✅ Rapport PDF généré : '{pdf_filename}'")
        return pdf_filename
    except Exception as e:
        # More specific error handling if needed
        print(f"❌ Erreur lors de la génération du PDF : {e}")
        return None

In [ ]:
pdf_file = None
if not df_metropoles.empty and CODE_COMMUNE_TEST in df_metropoles['code_commune'].unique():
    pdf_file = generate_pdf_report(CODE_COMMUNE_TEST, df_source=df_metropoles)
else:
    pdf_file = generate_pdf_report(CODE_COMMUNE_TEST) # Charge les données si besoin

# Afficher le lien de téléchargement si le fichier a été créé
if pdf_file and os.path.exists(pdf_file):
    display(FileLink(pdf_file))
else:
    print("Lien de téléchargement non disponible.")